In [1]:
import os
import json
import torch
import json
import ast
import argparse
import jsonlines

import pandas as pd
from peft import PeftModel,PeftConfig
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
user_token = os.getenv("HF_TOKEN")

model_name = "meta-llama/Llama-3.2-3B-Instruct"
adapter_name = "JesseLiu/llama32-3b-kpath"

/home/jesseliu/miniconda3/envs/dynh/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
peft_cfg = PeftConfig.from_pretrained(adapter_name, use_auth_token=user_token)
base_name = peft_cfg.base_model_name_or_path or model_name

tokenizer = AutoTokenizer.from_pretrained(adapter_name, use_auth_token=user_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
new_special_tokens = ['<degd>', '<ddd>', '<decgd>',
                    '<demgd>', '<debgd>', '<dppd>', '<dpd>']
tokenizer.add_special_tokens({'additional_special_tokens': new_special_tokens})

base_model = AutoModelForCausalLM.from_pretrained(
    base_name,
    torch_dtype="auto",
    device_map="auto",
    use_auth_token=user_token,
)

base_model.resize_token_embeddings(len(tokenizer))

model = PeftModel.from_pretrained(
    base_model,
    adapter_name,
    torch_dtype="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.14it/s]


In [3]:
print('<ddd>' in tokenizer.additional_special_tokens)  
print(tokenizer.convert_tokens_to_ids('<ddd>'))        
print(tokenizer.tokenize('<ddd>'))  

True
128257
['<ddd>']
